In [9]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

df=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
df_result=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv")   

df['bid_ask_spread']=df['ask']-df['bid']
df['trade']=df['trade'].astype(int)

value_categor=['venue','action','side']

for k in value_categor:

    df_encoded = pd.get_dummies(df[k], prefix=k).astype(int)
    df = df.merge( df_encoded , left_index = True , right_index = True)
    df.drop( columns=[k] , inplace = True )

df.drop(columns=['obs_id','order_id'],inplace=True)

value_no_center=['price' , 'bid' , 'ask' , 'bid_size', 'ask_size', 'bid_ask_spread' ]

for k in value_no_center:
    
    df[k] = df[k] / max( df[k] )

X=df
y=df_result[['eqt_code_cat']]
x_train, x_test, y_train, y_test = train_test_split(X, df_result['eqt_code_cat'].array, test_size=0.2, random_state=42)

KeyboardInterrupt: 

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Hypothèse : Les données sont dans les variables x_train et y_train, prétraitées selon la description.

# Définition du modèle
def create_benchmark_model():
    # Entrée : un tenseur de taille (100, 30)
    inputs = layers.Input(shape=(128640, 1900))

    # Deux cellules GRU : une en avant, une en arrière
    gru_fwd = layers.GRU(64, return_sequences=False)(inputs)
    gru_bwd = layers.GRU(64, return_sequences=False, go_backwards=True)(inputs)

    # Concaténation des sorties
    concatenated = layers.concatenate([gru_fwd, gru_bwd])

    # Couche dense de 128 vers 64 dimensions avec activation SeLU
    dense_1 = layers.Dense(64, activation='selu')(concatenated)

    # Couche dense de 64 vers 24 (softmax pour les probabilités des classes)
    output = layers.Dense(24, activation='softmax')(dense_1)

    # Création du modèle
    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    # Compilation du modèle avec Adam et une fonction de perte cross-entropy
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Création du modèle
model = create_benchmark_model()

# Entraînement du modèle sur 10 000 batchs
batch_size = 1000
epochs = 10  # ajustable selon les ressources disponibles

# Hypothèse : x_train et y_train sont prétraités et disponibles
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# Évaluation
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/10


ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 128640, 1900), found shape=(None, 1900)